# import librairies

In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler


In [19]:
df= pd.read_csv("climate_change_indicators.csv")
df.head()

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,...,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
0,1,"Afghanistan, Islamic Rep. of",AF,AFG,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",...,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.498,1.327,2.012
1,2,Albania,AL,ALB,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",...,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498,1.536,1.518
2,3,Algeria,DZ,DZA,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",...,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.926,2.330,1.688
3,4,American Samoa,AS,ASM,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",...,1.257,1.170,1.009,1.539,1.435,1.189,1.539,1.430,1.268,1.256
4,5,"Andorra, Principality of",AD,AND,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",...,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.562,1.533,3.243


In [20]:
# Sélectionner uniquement les colonnes numériques
df_numeric = df.select_dtypes(include=[np.number])

# Vérifier et traiter les valeurs manquantes (NaN)
print("Valeurs manquantes par colonne avant traitement :")
print(df_numeric.isna().sum())
df_numeric.fillna(0, inplace=True)  # Remplir les NaN avec 0

# Vérifier les colonnes à variance nulle
print("Variance par colonne :")
print(df_numeric.var())

# Supprimer les colonnes à variance nulle
df_numeric = df_numeric.loc[:, df_numeric.var() != 0]


Valeurs manquantes par colonne avant traitement :
ObjectId     0
F1961       37
F1962       36
F1963       37
F1964       37
            ..
F2018       12
F2019       12
F2020       13
F2021       12
F2022       12
Length: 63, dtype: int64
Variance par colonne :
ObjectId    4237.500000
F1961          0.140655
F1962          0.098083
F1963          0.125260
F1964          0.080544
               ...     
F2018          0.423060
F2019          0.312466
F2020          0.496070
F2021          0.313884
F2022          0.520814
Length: 63, dtype: float64


In [22]:
# Appliquer Min-Max Scaling pour normaliser les données
scaler = MinMaxScaler()
df_numeric_normalized = scaler.fit_transform(df_numeric)

# Convertir le DataFrame normalisé en DataFrame Pandas pour la manipulation
df_numeric_normalized = pd.DataFrame(df_numeric_normalized, columns=df_numeric.columns)

# Vérification de la taille du DataFrame après traitement
print(f"Le DataFrame contient {len(df_numeric_normalized)} lignes après traitement.")

Le DataFrame contient 225 lignes après traitement.


In [23]:
# Fonction pour préparer les séquences pour le RNN
def create_sequences(df, sequence_length):
    sequences = []
    labels = []
    for i in range(len(df) - sequence_length):
        sequences.append(df.iloc[i:i + sequence_length].values)
        labels.append(df.iloc[i + sequence_length].values)
    return np.array(sequences), np.array(labels)

sequence_length = 30
X, y = create_sequences(df_numeric_normalized, sequence_length)

In [25]:
# Définir le modèle RNN
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(y.shape[1])  # Prédiction de plusieurs variables (nombre de colonnes dans y)
])
# Compiler le modèle
model.compile(optimizer='adam', loss='mse')

In [26]:
# Convertir les données en float32
X = X.astype(np.float32)
y = y.astype(np.float32)

# Entraînement du modèle
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

# Prédiction
future_predictions = model.predict(X[-1].reshape(1, sequence_length, X.shape[2]))

# Afficher les prédictions futures
print("Prédictions futures :")
print(future_predictions)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - loss: 0.2694 - val_loss: 0.2255
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2335 - val_loss: 0.2032
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.2034 - val_loss: 0.1786
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.1800 - val_loss: 0.1518
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1486 - val_loss: 0.1275
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.1274 - val_loss: 0.1041
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.1057 - val_loss: 0.0844
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0850 - val_loss: 0.0681
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0668 - val_loss: 0.0553
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0586 - val_loss: 0.0465
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0483 - val_loss: 0.0405
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0398 - val_loss: 0.0361
E

In [99]:
from qiskit import transpile, QuantumCircuit
from qiskit.circuit.library import TwoLocal
from qiskit_aer import Aer, AerSimulator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

In [86]:
# un circuit quantique de base
def create_quantum_circuit(params):
    qc = QuantumCircuit(2)
    qc.h([0, 1])
    qc.cx(0, 1)
    # Ajouter des paramètres au circuit
    qc.ry(params[0], 0)
    qc.ry(params[1], 1)
    return qc

# l'instance quantique et l'optimiseur
backend = AerSimulator()  # Utilisez AerSimulator pour la simulation

# la forme de variation
var_form = TwoLocal(num_qubits=2, entanglement='full')
optimizer = COBYLA()


In [101]:
# fonction objectif pour VQE
def objective_function(params):
    qc = create_quantum_circuit(params)
    transpiled_circuit = transpile(qc, backend)
    result = backend.run(transpiled_circuit).result()
    counts = result.get_counts()
    # Calculer une mesure des résultats 
    return -counts.get('00', 0)  

# Configurer le VQE 
vqe = VQE(ansatz=var_form, optimizer=optimizer, estimator=Aer.get_backend('statevector_simulator'))